<a href="https://colab.research.google.com/github/Coreight98/Paran_ajou_Project_Anything/blob/main/Pattern_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import time

In [ ]:
train_location = './drive/MyDrive/data/DTD/'
test_location = './drive/MyDrive/data/DTD_test/'

X = np.empty((0, 4), dtype=int)
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3,stride=1, padding=1),
            torch.nn.BatchNorm2d(num_features=32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=4, stride=4))

        # 두번째층
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3,stride=1, padding=1),
            torch.nn.BatchNorm2d(num_features=64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            torch.nn.ReLU(),
            torch.nn.AvgPool2d(kernel_size=2, stride=2))
        
        # 세번째층
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 256, kernel_size=3,stride=1, padding=1),
            torch.nn.BatchNorm2d(num_features=256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            torch.nn.ReLU(),
            torch.nn.AvgPool2d(kernel_size=2, stride=2))
        
        # 네번째층
        self.layer4 = torch.nn.Sequential(
            torch.nn.Conv2d(256, 512, kernel_size=3,stride=1, padding=1),
            torch.nn.BatchNorm2d(num_features=512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            torch.nn.ReLU(),
            torch.nn.AvgPool2d(kernel_size=2, stride=2))
        
        # 다섯번째층
        self.layer5 = torch.nn.Sequential(
            torch.nn.Conv2d(512, 1024, kernel_size=3,stride=1, padding=1),
            torch.nn.BatchNorm2d(num_features=1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            torch.nn.ReLU(),
            torch.nn.AvgPool2d(kernel_size=2, stride=2))

        # 6번째층
        self.layer6 = torch.nn.Sequential(
            torch.nn.Conv2d(1024, 2048, kernel_size=3,stride=1, padding=1),
            torch.nn.BatchNorm2d(num_features=2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            torch.nn.ReLU(),
            torch.nn.AvgPool2d(kernel_size=2, stride=2))
       
        # 전결합층 
        self.fc = torch.nn.Linear(2048, 38, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        global X
        out = self.layer1(x)
        # print(out.shape)
        out = self.layer2(out)
        # print(out.shape)
        out = self.layer3(out)
        # print(out.shape)
        out = self.layer4(out)
        # print(out.shape)
        out = self.layer5(out)
        # print(out.shape)
        out = self.layer6(out)
        # print(out.shape)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        X = np.append(X,out.detach().numpy())
        return out

class PatternDataset():
    def __init__(self, image, class_to_int, mode, transforms):
        super().__init__()
        self.image = image
        self.class_to_int = class_to_int
        self.mode = mode
        self.transforms = transforms

    def __getitem__(self, index):

        if self.mode == 'train' :
            image_name = self.image[index]
            image = Image.open(train_location + image_name)
            image = image.resize((224,224))
            #레이블 입력
            label = self.class_to_int[image_name.split('_')[0]]
            label = torch.tensor(label,dtype=torch.long)
            #이미지 변형 적용
            image = self.transforms(image)
            return image, label
        elif self.mode == 'test':
            image_name = self.image[index]
            image = Image.open(test_location + image_name)
            image = image.resize((224,224))
            #레이블 입력
            label = self.class_to_int[image_name.split('_')[0]]
            label = torch.tensor(label,dtype=torch.long)
            #이미지 변형 적용용
            image = self.transforms(image)
            return image, label

    def __len__(self):
        return len(self.image)

def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
PATH = './drive/MyDrive/DTD_save/net1.pth'
net = torch.load(PATH)
net.eval()
X = np.load('./drive/MyDrive/DTD_save/X1.npy')

img=PIL.Image.open('./drive/MyDrive/DTD_test/???.jpg')
img = img.resize((224,224))
emp = torch.empty(1,1,224,224)
img_t =transform(img)
emp[0] = img_t
input = net(emp)

X = np.reshape(X,(1281,5)) #1281개 5개tensor
Y = np.ones((1281,1),dtype=int)

reg = KNeighborsClassifier(n_neighbors=4)
reg.fit(X,Y)
l = reg.kneighbors(input.detach().numpy(),n_neighbors=4,return_distance=False)

l.sort()
print(l)

for i in l:
  temp = np.empty((0,4),dtype=int)
  temp = np.append(temp, X[i])
temp = np.reshape(temp,(4,5))

dataiter = iter(trainloader)

count = 0

index=0

images = torch.empty(4,1,224,224)


while(index<4):
  if count < l[0][index]:
    imgg=img
    img=dataiter.next()
    count+=4
  else:
    print("find!")
    #l[0][index]-count 한 값을 인덱스로 dataiter에서 출력
    k=count-l[0][index]
    print(k)
    if k == 0:
      images[index][0] = img[0][k][0]
    else:
      images[index][0] = imgg[0][4-k][0]
    count+=4
    index+=1
    if index != 4:
      dataiter.next()
    
#이미지 출력
imshow(torchvision.utils.make_grid(images))

from IPython.display import Image 
img=Image('./drive/MyDrive/DTD_test/???.jpg')
display(img)
from PIL import Image